In [173]:
import re
import pickle
from glob import glob
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras import Model
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Flatten, Dense
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import np_utils

In [174]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
project_dir = "gdrive/My Drive/metis_project_5"

In [4]:
# Get easy labels

# with open('labels/classifier/easy_labels.csv') as f:
#     raw_easy_labels = f.read().split('\n')[1:-1]

# easy_label_regex = re.compile('^([^\d]*)([^.]*)\.png,(.*)$')

# easy_label_to_int = {'a': 0, 'g': 1, 'db': 2, 'ub': 3, 's': 4}

# easy_paths, easy_y = [], []
# for label in raw_easy_labels:
#     (category, frame, label) = label_regex.match(label).groups()
#     int_label = label_to_int.get(label)
#     if int_label and category == 'new_fox_':
#         path = f"data/larger_cropped/{frame}.png"
#         easy_paths.append(path)
#         easy_y.append(int_label)

# easy_num_categories = 5

In [29]:
# Get new easy_larger_cropped labels, wiz_sfat

with open(f"{project_dir}/labels/classifier/easy_wiz_sfat_larger_cropped_labels.csv") as f:
    raw_ws_labels = f.read().split('\n')[1:-1]

ws_label_regex = re.compile('^([^.]*)\.png,(.*)$')

ws_label_to_int = {
    'g': 0,
    'a': 1,
    's': 0,
    'aub': 1,
    'gdb': 0,
    'adb': 1,
    'l': 1,
}
ws_num_categories = 2

ws_paths, ws_labels, ws_y = [], [], []
for label in raw_ws_labels:
    frame, label = ws_label_regex.match(label).groups()
    if label in ws_label_to_int.keys():
        int_label = ws_label_to_int[label]
        if int_label:
            ws_labels.append('a')
        else:
            ws_labels.append('g')
        path = f"{project_dir}/data/wiz_sfat/larger_cropped/{frame}.png"
        ws_paths.append(path)
        ws_y.append(int_label)

print(Counter(ws_labels))

Counter({'g': 212, 'a': 136})


In [138]:
# Get semi- easy_larger_cropped labels, wiz_sfat

with open(f"{project_dir}/labels/classifier/easy_wiz_sfat_larger_cropped_labels.csv") as f:
    raw_se_labels = f.read().split('\n')[1:-1]

se_label_regex = re.compile('^([^.]*)\.png,(.*)$')

se_label_to_int = {
    'g': 0,
    'a': 1,
    's': 2,
    'aub': 3,
    'gdb': 4,
    'adb': 4,
    'l': 5,
}
se_num_categories = 6

se_paths, se_labels, se_y = [], [], []
for label in raw_se_labels:
    frame, label = se_label_regex.match(label).groups()
    se_labels.append(label)
    if label in se_label_to_int.keys():
        int_label = se_label_to_int[label]
        path = f"{project_dir}/data/wiz_sfat/larger_cropped/{frame}.png"
        se_paths.append(path)
        se_y.append(int_label)

print(Counter(se_labels))

Counter({'g': 200, 'a': 128, 's': 7, 'gdb': 5, 'aub': 5, 'adb': 2, 'l': 1})


In [198]:
# Get hard labels, wiz_sfat

with open(f"{project_dir}/labels/classifier/hard_wiz_sfat_labels.csv") as f:
    raw_hws_labels = f.read().split('\n')[1:-1]

hws_label_regex = re.compile('^([^\d]*)([^.]*)\.png,.,(.*)$')

hws_label_to_int = {
    'a': 0,
    'adb': 1,
    'anb': 2,
    'aub': 3,
    'ba': 4,
    'd': 5,
    'da': 6,
    'dt': 7,
    'g': 8,
    'gdb': 9,
    'h': 10,
    'ha': 12,
    'hg': 12,
    'j': 13,
    'l': 14,
    'na': 15,
    'p': 16,
    's': 17,
    'sd': 18,
    't': 19,
    'ua': 20,
    'ut': 21,
}
hws_num_categories = 22

hws_paths, hws_labels, hws_y = [], [], []
for label in raw_hws_labels:
    (category, frame, label) = hws_label_regex.match(label).groups()
    int_label = hws_label_to_int.get(label)
    hws_labels.append(label)
    if int_label and category == 'new_fox_':
        int_label = hws_label_to_int[label]
        path = f"{project_dir}/data/wiz_sfat/larger_cropped/{frame}.png"
        hws_paths.append(path)
        hws_y.append(int_label)

print(Counter(hws_labels))

Counter({'g': 263, 'a': 175, 'ha': 20, 'ba': 15, 'na': 14, 'ut': 9, 'gdb': 7, 's': 7, 'd': 6, 'da': 5, 'aub': 5, 'hg': 3, 'l': 3, 'ub': 3, 'adb': 2, 'anb': 2, 'ua': 2, 'h': 2, 't': 1, 'p': 1, 'sd': 1, 'dt': 1, 'j': 1})


In [8]:
# Get easy labels, tbh7_purp_fox_vgbc

with open(f"{project_dir}/labels/classifier/easy_tbh7_purp_fox_vgbc_labels.csv") as f:
    raw_tbh_labels = f.read().split('\n')[1:-1]

tbh_label_regex = re.compile('^([^.]*)\.png,(.*)$')

tbh_label_to_int = {
    'g': 0,
    'a': 1,
    's': 0,
    'aub': 1,
    'gdb': 0,
    'adb': 1,
    'l': 1,
}
tbh_num_categories = 2

# tbh_label_to_int = {
#     'g': 0,
#     'a': 1,
#     's': 2,
#     'aub': 3,
#     'gdb': 4,
#     'adb': 5,
#     'l': 6,
# }
# tbh_num_categories = 7

tbh_paths, tbh_labels, tbh_y = [], [], []
for label in raw_tbh_labels:
    frame, label = tbh_label_regex.match(label).groups()
    tbh_labels.append(label)
    if label in tbh_label_to_int.keys():
        int_label = tbh_label_to_int[label]
        path = f"{project_dir}/data/tbh7_purp_fox_vgbc/larger_cropped/{frame}.png"
        tbh_paths.append(path)
        tbh_y.append(int_label)

print(Counter(tbh_labels))

Counter({'g': 187, 'a': 173, 'aub': 18, 'l': 10, 's': 6, 'adb': 3, 'gdb': 3})


In [26]:
img_hw = (350, 350)
img_dimensions = (350, 350, 3)

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=img_hw)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    # x = mobilenet_v2.preprocess_input(x)
    return x

In [27]:
# Prepare test/train data, wiz_sfat easy

ws_X = np.array([prepare_image(path)[0] for path in ws_paths])

ws_X_tr, ws_X_te, ws_y_tr, ws_y_te = train_test_split(ws_X, ws_y, test_size=0.2, random_state=42)

In [139]:
# Prepare test/train data, wiz_sfat semi-easy

se_X = np.array([prepare_image(path)[0] for path in se_paths])

se_X_tr, se_X_te, se_y_tr, se_y_te = train_test_split(se_X, se_y, test_size=0.2, random_state=42)

In [199]:
# Prepare test/train data, wiz_sfat hard

hws_X = np.array([prepare_image(path)[0] for path in hws_paths])

hws_X_tr, hws_X_te, hws_y_tr, hws_y_te = train_test_split(hws_X, hws_y, test_size=0.2, random_state=42)

In [ ]:
# Prepare test/train data, tbh7

tbh_X = np.array([prepare_image(path)[0] for path in tbh_paths])

tbh_X_tr, tbh_X_te, tbh_y_tr, tbh_y_te = train_test_split(tbh_X, tbh_y, test_size=0.2, random_state=42)

In [32]:
pickles_dir = project_dir + "/pickles"

!ls gdrive/My\ Drive/metis_project_5/pickles

model_1.pickle	 tbh_y_tr_cat.pickle  ws_y_te.pickle	  X_tr.pickle
tbh_X_te.pickle  tbh_y_tr.pickle      ws_y_tr_cat.pickle  y_te.pickle
tbh_X_tr.pickle  ws_X_te.pickle       ws_y_tr.pickle	  y_tr_cat.pickle
tbh_y_te.pickle  ws_X_tr.pickle       X_te.pickle	  y_tr.pickle


In [ ]:
# Dump easy wiz_sfat data

# with open(f'{pickles_dir}/ws_X_tr.pickle', 'wb') as f:
#     pickle.dump(ws_X_tr, f)

# with open(f'{pickles_dir}/ws_X_te.pickle', 'wb') as f:
#     pickle.dump(ws_X_te, f)

# with open(f'{pickles_dir}/ws_y_tr.pickle', 'wb') as f:
#     pickle.dump(ws_y_tr, f)

# with open(f'{pickles_dir}/ws_y_te.pickle', 'wb') as f:
#     pickle.dump(ws_y_te, f)

In [168]:
# Load easy wiz_sfat data

with open(f'{pickles_dir}/ws_X_tr.pickle', 'rb') as f:
    ws_X_tr = pickle.load(f)

with open(f'{pickles_dir}/ws_X_te.pickle', 'rb') as f:
    ws_X_te = pickle.load(f)

with open(f'{pickles_dir}/ws_y_tr.pickle', 'rb') as f:
    ws_y_tr = pickle.load(f)

with open(f'{pickles_dir}/ws_y_te.pickle', 'rb') as f:
    ws_y_te = pickle.load(f)

In [140]:
# Dump semi-easy wiz_sfat data

# with open(f'{pickles_dir}/se_X_tr.pickle', 'wb') as f:
#     pickle.dump(se_X_tr, f)

# with open(f'{pickles_dir}/se_X_te.pickle', 'wb') as f:
#     pickle.dump(se_X_te, f)

# with open(f'{pickles_dir}/se_y_tr.pickle', 'wb') as f:
#     pickle.dump(se_y_tr, f)

# with open(f'{pickles_dir}/se_y_te.pickle', 'wb') as f:
#     pickle.dump(se_y_te, f)

In [ ]:
# Load semi-easy wiz_sfat data

with open(f'{pickles_dir}/se_X_tr.pickle', 'rb') as f:
    se_X_tr = pickle.load(f)

with open(f'{pickles_dir}/se_X_te.pickle', 'rb') as f:
    se_X_te = pickle.load(f)

with open(f'{pickles_dir}/se_y_tr.pickle', 'rb') as f:
    se_y_tr = pickle.load(f)

with open(f'{pickles_dir}/se_y_te.pickle', 'rb') as f:
    se_y_te = pickle.load(f)

In [200]:
# Dump hard wiz_sfat data

# with open(f'{pickles_dir}/hws_X_tr.pickle', 'wb') as f:
#     pickle.dump(hws_X_tr, f)

# with open(f'{pickles_dir}/hws_X_te.pickle', 'wb') as f:
#     pickle.dump(hws_X_te, f)

# with open(f'{pickles_dir}/hws_y_tr.pickle', 'wb') as f:
#     pickle.dump(hws_y_tr, f)

# with open(f'{pickles_dir}/hws_y_te.pickle', 'wb') as f:
#     pickle.dump(hws_y_te, f)

In [201]:
# Load hard wiz_sfat data

with open(f'{pickles_dir}/hws_X_tr.pickle', 'rb') as f:
    hws_X_tr = pickle.load(f)

with open(f'{pickles_dir}/hws_X_te.pickle', 'rb') as f:
    hws_X_te = pickle.load(f)

with open(f'{pickles_dir}/hws_y_tr.pickle', 'rb') as f:
    hws_y_tr = pickle.load(f)

with open(f'{pickles_dir}/hws_y_te.pickle', 'rb') as f:
    hws_y_te = pickle.load(f)

In [ ]:
# Dump easy tbh data

# with open(f'{pickles_dir}/tbh_X_tr.pickle', 'wb') as f:
#     pickle.dump(tbh_X_tr, f)

# with open(f'{pickles_dir}/tbh_X_te.pickle', 'wb') as f:
#     pickle.dump(tbh_X_te, f)

# with open(f'{pickles_dir}/tbh_y_tr.pickle', 'wb') as f:
#     pickle.dump(tbh_y_tr, f)

# with open(f'{pickles_dir}/tbh_y_te.pickle', 'wb') as f:
#     pickle.dump(tbh_y_te, f)

In [170]:
# Load easy tbh data

with open(f'{pickles_dir}/tbh_X_tr.pickle', 'rb') as f:
    tbh_X_tr = pickle.load(f)

with open(f'{pickles_dir}/tbh_X_te.pickle', 'rb') as f:
    tbh_X_te = pickle.load(f)

with open(f'{pickles_dir}/tbh_y_tr.pickle', 'rb') as f:
    tbh_y_tr = pickle.load(f)

with open(f'{pickles_dir}/tbh_y_te.pickle', 'rb') as f:
    tbh_y_te = pickle.load(f)

In [ ]:
# Choose actual training and test sets

In [144]:
# Train on semi-easy labels with wiz_sfat images
X_tr = se_X_tr
y_tr = se_y_tr
X_te = se_X_te
y_te = se_y_te
num_categories = se_num_categories

In [171]:
# Train on tbh images (larger set) and test on wiz_sfat images
X_tr = np.concatenate([tbh_X_tr, tbh_X_te])
y_tr = tbh_y_tr + tbh_y_te
X_te, y_te = ws_X_te, ws_y_te
num_categories = tbh_num_categories

In [ ]:
# Train on both train sets and test on both test sets
X_tr = np.concatenate([tbh_X_tr, ws_X_tr])
y_tr = tbh_y_tr + ws_y_tr
X_te = np.concatenate([tbh_X_te, ws_X_te])
y_te = tbh_y_te + ws_y_te
num_categories = ws_num_categories

In [202]:
# Train on hard labels with wiz_sfat images
X_tr = hws_X_tr
y_tr = hws_y_tr
X_te = hws_X_te
y_te = hws_y_te
num_categories = hws_num_categories

In [203]:
# If multi-class, make categorical targets
if num_categories > 2:
  y_tr_cat_compare = np_utils.to_categorical(y_tr)
  num_tr = len(y_tr)
  y = np.concatenate([y_tr, y_te])
  y_cat = np_utils.to_categorical(y)
  y_tr_cat = y_cat[:num_tr]
  y_te_cat = y_cat[num_tr:]
  assert(len(np.concatenate([y_tr_cat, y_te_cat])) == len(y))
  assert(len(y_tr) == len(y_tr_cat))
  assert(len(y_te) == len(y_te_cat))
  assert((y_tr_cat == y_tr_cat_compare).all())
else:
  y_tr_cat = y_tr
  y_te_cat = y_te

In [206]:
# Fetch pretrained model

pretrained = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=img_dimensions,
)

In [207]:
# Build model

for layer in pretrained.layers[:17]:
    layer.trainable = False

model = Sequential()
model.add(pretrained)
model.add(Flatten())

if num_categories > 2:
    model.add(Dense(num_categories))
    loss='categorical_crossentropy',
else:
    model.add(Dense(1, activation='sigmoid'))
    loss='binary_crossentropy',

model.compile(
    loss=loss,
    optimizer='adam',
    metrics=['accuracy'],
)

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 10, 10, 512)       14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 22)                1126422   
Total params: 15,841,110
Trainable params: 3,486,230
Non-trainable params: 12,354,880
_________________________________________________________________


In [208]:
# Image augmentation
tr_datagen = image.ImageDataGenerator()
    # rescale=1./255,
    # featurewise_std_normalization=True,
    # vertical_flip=True,
    # validation_split=0.25)

In [209]:
# Only applying scaling to the validation set
# te_datagen = ImageDataGenerator(rescale=1./255)
te_datagen = tr_datagen

In [210]:
# Training and test gerators
tr_gen = tr_datagen.flow(X_tr, y_tr_cat)
te_gen = te_datagen.flow(X_te, y_te_cat)
epochs=100

# Callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=30, verbose=1,
                   restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.2,
                   patience=10, cooldown=5, min_lr=0.0002)

In [211]:
# Train model
model.fit_generator(tr_gen, steps_per_epoch=10, epochs=epochs, callbacks=[es, reduce_lr], 
                 verbose=1, validation_data=te_gen, validation_steps=20)

Epoch 1/100
10/10 [==============================] - 26s 3s/step - loss: 9.2231 - accuracy: 0.3979 - val_loss: 12.2476 - val_accuracy: 0.6923
Epoch 2/100
10/10 [==============================] - 19s 2s/step - loss: 13.1492 - accuracy: 0.6523 - val_loss: 14.0881 - val_accuracy: 0.6923
Epoch 3/100
10/10 [==============================] - 19s 2s/step - loss: 12.3601 - accuracy: 0.6303 - val_loss: 13.3536 - val_accuracy: 0.6923
Epoch 4/100
10/10 [==============================] - 19s 2s/step - loss: 12.7653 - accuracy: 0.6391 - val_loss: 11.7866 - val_accuracy: 0.6923
Epoch 5/100
10/10 [==============================] - 19s 2s/step - loss: 9.6285 - accuracy: 0.6408 - val_loss: 0.8059 - val_accuracy: 0.6923
Epoch 6/100
10/10 [==============================] - 20s 2s/step - loss: 10.6542 - accuracy: 0.6313 - val_loss: 2.4177 - val_accuracy: 0.6923
Epoch 7/100
10/10 [==============================] - 19s 2s/step - loss: 4.1013 - accuracy: 0.6444 - val_loss: 4.0295 - val_accuracy: 0.6923
Epoch

KeyboardInterrupt: ignored

In [129]:
# Train model

for _ in range(1):
  if num_categories > 2:
      model.fit(X_tr, y_tr_cat)
  else:
      model.fit(X_tr, y_tr)
      y_pr = model.predict_classes(X_te)[:,0]
      print(f"Test accuracy: {len(y_pr[y_pr == y_te]) / len(y_pr)}")

Epoch 1/1
278/278 [==============================] - 7s 24ms/step - loss: 9.5555 - accuracy: 0.5504


In [77]:
# Save / load model

models_dir = project_dir + "/models"
!mkdir -p gdrive/My\ Drive/metis_project_5/models

models = glob(models_dir + '/*')
latest_model_num = int(re.sub(".*model_(\d*)\..*", "\\1", sorted(models)[-1]))
next_model_num = latest_model_num + 1

In [120]:
model.save_weights(f'{models_dir}/model_{next_model_num}.hdf5')

In [164]:
model.load_weights(f'{models_dir}/model_{next_model_num}.hdf5')
# model.load_weights(f'{models_dir}/model_{latest_model_num}.hdf5')

In [126]:
pretrained.weights[0][0][0][0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.00040788297>

In [130]:
pretrained.weights[0][0][0][0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=-0.0012804723>

In [ ]:
print(f"Train zeros: {len(list(filter(lambda x: x == 0, y_tr)))}")
print(f"Train ones: {len(list(filter(lambda x: x == 1, y_tr)))}")
print(f"Test zeros: {len(list(filter(lambda x: x == 0, y_te)))}")
print(f"Test ones: {len(list(filter(lambda x: x == 1, y_te)))}")

Train zeros: 168
Train ones: 109
Test zeros: 44
Test ones: 26


In [52]:
# for x in zip(y_pr, y_te):
#   print(x)

In [ ]:
# Predict and score

y_pr = model.predict_classes(X_te)[:,0]

print(f"Test accuracy: {len(y_pr[y_pr == y_te]) / len(y_pr)}")

assert(len(y_pr) == len(y_te))

y_tr_pr = model.predict_classes(X_tr)[:,0]

print(f"Train accuracy: {len(y_tr_pr[y_tr_pr == y_tr]) / len(y_tr_pr)}")

assert(len(y_tr_pr) == len(y_tr))

Test accuracy: 0.8285714285714286
Train accuracy: 0.9566787003610109


In [53]:
# print(y_pr)
# print(y_te)
# print(y_tr_pr)
# print(y_tr)